In [1]:
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
import time
import pydde as d

In [2]:
#Parameters
samplenum = 10
epochs = 10
hiddenlayers = [90]
input_size = 3
output_size = 3
learning_rate = 0.001
time_length = 60; #seconds

In [3]:
# Generate simulation
dyn = d.PyDyn('Data/point-mass_pendulum.sim', time_length)
state_init = dyn.compute(dyn.p_init)
f = dyn.f(state_init, dyn.p_init)
df = dyn.df_dp(state_init, dyn.p_init)
dy = dyn.dy_dp(state_init, dyn.p_init)

In [4]:
#Sample targets only variables in z direction
y_target = np.zeros((samplenum,3))
y_target[:,2] = np.random.rand(samplenum)
#x[:,0] = np.random.rand(samplenum)
y_target[:,1] = 2
y_target= torch.tensor(y_target)

## Building the custon Simulation Activation Function

In [50]:
import torch

class LinearFunction(torch.autograd.Function):

    @staticmethod
    # bias is an optional argument
    def forward(ctx, input, weight):
        ctx.save_for_backward(input, weight)
        output = input.t().mm(weight).t()
        print(f'output: {output}')

        return output

    # This function has only a single output, so it gets only one gradient
    @staticmethod
    def backward(ctx, grad_output):
        # This is a pattern that is very convenient - at the top of backward
        # unpack saved_tensors and initialize all gradients w.r.t. inputs to
        # None. Thanks to the fact that additional trailing Nones are
        # ignored, the return statement is simple even when the function has
        # optional inputs.
        print(f'grad_output: {grad_output}')
        input, weight = ctx.saved_tensors
        #grad_input = grad_weight = None

        # These needs_input_grad checks are optional and there only to
        # improve efficiency. If you want to make your code simpler, you can
        # skip them. Returning gradients for inputs that don't require it is
        # not an error.
        grad_input = grad_output.t().mm(weight.t())
        grad_weight = grad_output.mm(input.t()).t()
        #print(f'grad_input: {grad_input.shape}')
        #print(f'grad_weight: {grad_weight.shape}')
        return grad_input, grad_weight

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 10, 3, 100, 60
learning_rate = 0.01
# Create random Tensors to hold input and outputs.
x = torch.ones(D_in, N)
y = torch.randn(D_out, N)

# Create random Tensors for weights.
w1 = torch.ones(D_in, D_out, requires_grad=True)
#w2 = torch.randn(H, D_out, requires_grad=True)


learning_rate = 1e-6


In [53]:
for t in range(10):
    # To apply our Function, we use Function.apply method. We alias this as 'relu'.
    Lin = LinearFunction.apply

    # Forward pass: compute predicted y using operations; we compute
    # ReLU using our custom autograd operation.
    y_pred = Lin(x, w1)

    # Compute and print loss
    #loss = (y_pred - y).pow(2).sum()
    loss = y_pred.sum() * 100
    print(f'grad of y_pred: {y_pred.requires_grad}')
    print(t, loss.item())
    # Use autograd to compute the backward pass.
    loss.backward(retain_graph=True)
    # Update weights using gradient descent
    with torch.no_grad():
        w1 -= learning_rate * w1.grad

        # Manually zero the gradients after updating weights
        w1.grad.zero_()

2.9160],
        [2.9160, 2.9160, 2.9160, 2.9160, 2.9160, 2.9160, 2.9160, 2.9160, 2.9160,
         2.9160],
        [2.9160, 2.9160, 2.9160, 2.9160, 2.9160, 2.9160, 2.9160, 2.9160, 2.9160,
         2.9160],
        [2.9160, 2.9160, 2.9160, 2.9160, 2.9160, 2.9160, 2.9160, 2.9160, 2.9160,
         2.9160],
        [2.9160, 2.9160, 2.9160, 2.9160, 2.9160, 2.9160, 2.9160, 2.9160, 2.9160,
         2.9160],
        [2.9160, 2.9160, 2.9160, 2.9160, 2.9160, 2.9160, 2.9160, 2.9160, 2.9160,
         2.9160],
        [2.9160, 2.9160, 2.9160, 2.9160, 2.9160, 2.9160, 2.9160, 2.9160, 2.9160,
         2.9160],
        [2.9160, 2.9160, 2.9160, 2.9160, 2.9160, 2.9160, 2.9160, 2.9160, 2.9160,
         2.9160],
        [2.9160, 2.9160, 2.9160, 2.9160, 2.9160, 2.9160, 2.9160, 2.9160, 2.9160,
         2.9160],
        [2.9160, 2.9160, 2.9160, 2.9160, 2.9160, 2.9160, 2.9160, 2.9160, 2.9160,
         2.9160],
        [2.9160, 2.9160, 2.9160, 2.9160, 2.9160, 2.9160, 2.9160, 2.9160, 2.9160,
         2.9160],
 

## Building the Model

In [6]:
class ActiveLearn(nn.Module):

    def __init__(self, n_in, out_sz):
        super(ActiveLearn, self).__init__()

        self.L_in = nn.Linear(n_in, hiddenlayers[0])
        self.H1 = nn.Linear(hiddenlayers[0], 3*time_length)
        #self.H1 = nn.Linear(hiddenlayers[0], hiddenlayers[1])
        #self.H2 = nn.Linear(hiddenlayers[1], 3*time_length)
        self.P = nn.Linear(3*time_length, 3*time_length)
        self.Relu = nn.ReLU(inplace=True)
    
    def forward(self, input):
        x = self.L_in(input)
        x = self.Relu(x)
        x = self.H1(x)
        x = self.Relu(x)
        #x = self.H2(x)
        #x = self.Relu(x)
        x = self.P(x)
        return x


model = ActiveLearn(input_size, output_size)

criterion = nn.MSELoss()  # RMSE = np.sqrt(MSE)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

y_target = y_target.float()

## Train the model

In [8]:
torch.autograd.set_detect_anomaly(True)

start_time = time.time()
weight_c1 = 1 # p start condition
weight_c2 = 0.1 # p smoothness condition
losses = []
y_preds= np.zeros((samplenum, 3))
p_preds= np.zeros((samplenum, 3*time_length))

#y_pred = torch.tensor(y_pred)
for i in range(epochs):
    for s in range(samplenum):
        y_truth = y_target[s, :]
        p_pred = model(y_truth)
        y_pred = Simulate(p_pred)
        y_preds[s, :] = y_pred.detach()
        p_preds[s, :] = p_pred.detach()
        #smoothness_error = Smooth(p_pred)
        error = 0
        smoothness_error = criterion(p_pred[0:3*(time_length-1)], p_pred[3:3*time_length])
        loss = torch.sqrt(criterion(y_pred.float(), y_truth)) # RMSE
        #loss = criterion(y_pred.float(), y_truth) + weight_c1*(sum(p_pred[0:3]-torch.tensor(dyn.p_init[0:3])))**2  # MSE + start condition penalty
        #loss = criterion(y_pred.float(), y_truth) + weight_c1*criterion(p_pred[0:3].double(), torch.tensor(dyn.p_init[0:3])) + weight_c2*smoothness_error# MSE + start condition penalty + p smoothness condition penalty
        #loss =sum((y_pred.float()-y_truth)**2) 
        losses.append(loss)
        optimizer.zero_grad()
        #Back Prop
        loss.backward()
        optimizer.step()
    print(f'epoch: {i:3}/{epochs}  loss: {loss.item():10.8f}')

print(f'epoch: {i:3} loss: {loss.item():10.8f}') # print the last line
print(f'\nDuration: {(time.time() - start_time)/60:.3f} min') # print the time elapsed

epoch:   0/100  loss: 4.29265270
epoch:   1/100  loss: 1.59549517
epoch:   2/100  loss: 1.19569693
epoch:   3/100  loss: 1.06783164
epoch:   4/100  loss: 0.71800677
epoch:   5/100  loss: 0.34746730
epoch:   6/100  loss: 0.24957020
epoch:   7/100  loss: 0.14124617
epoch:   8/100  loss: 0.05908810
epoch:   9/100  loss: 0.06164659
epoch:  10/100  loss: 0.04696999
epoch:  11/100  loss: 0.04116096
epoch:  12/100  loss: 0.03364264
epoch:  13/100  loss: 0.03126724
epoch:  14/100  loss: 0.02766821
epoch:  15/100  loss: 0.02761018
epoch:  16/100  loss: 0.02515688
epoch:  17/100  loss: 0.02508319
epoch:  18/100  loss: 0.02201077
epoch:  19/100  loss: 0.10844367
epoch:  20/100  loss: 0.07579721
epoch:  21/100  loss: 0.06616149
epoch:  22/100  loss: 0.07300152
epoch:  23/100  loss: 0.12004107
epoch:  24/100  loss: 0.10938503
epoch:  25/100  loss: 0.15107416
epoch:  26/100  loss: 0.14625780
epoch:  27/100  loss: 0.13239175
epoch:  28/100  loss: 0.05033869
epoch:  29/100  loss: 0.05827785
epoch:  30

## Test forward propagation

In [16]:
y_target= torch.tensor([0, 2, 0.5])
p = model(y_target)
p = p.detach().numpy()

y_pred_state = dyn.compute(p)
y_pred = y_pred_state.y[-3:]

print(y_pred)
print(y_target)
print(f'p trajecory: {p}')
print(f"y trajectory: {y_pred_state.y}")
error = 0
for i in range(time_length-1):
    step = sum((p[3*i:3*i+3] - p[3*i+3:3*i+6])**2)
    error = error + step
print(error/time_length)


[0.00338275 1.9851979  0.530538  ]
tensor([0.0000, 2.0000, 0.5000])
p trajecory: [-2.83841975e-03  2.99166989e+00  5.00382856e-04 -4.57676828e-01
  2.63269997e+00  5.21992743e-01 -6.81326151e-01  2.34795642e+00
 -2.74650395e-01 -5.83646119e-01  1.52864778e+00 -5.56134224e-01
 -4.80328590e-01  1.39862609e+00 -3.67586166e-01  2.40416359e-02
  4.23111081e-01  5.87768704e-02 -2.33206809e-01  4.84932035e-01
 -1.22672468e-01  2.28466719e-01  2.91416436e-01  2.32087135e-01
  2.46739164e-01  9.78568010e-03  3.55454922e-01  1.80878788e-01
  7.32885599e-02  4.53660607e-01  1.05677485e-01  8.67701948e-01
  5.26671529e-01  1.63849890e-01  4.57568586e-01  3.87044370e-01
  1.79882601e-01 -3.42988282e-01  2.72964060e-01  8.99747312e-02
  3.77623886e-01  1.89842135e-01  7.42183533e-03  3.33228946e-01
 -1.25915036e-01 -5.38735166e-02  6.11900687e-01 -4.17560935e-01
 -2.79457569e-01 -4.41163778e-01 -1.97180994e-02 -3.17232817e-01
 -2.63173133e-01 -6.54147416e-02 -2.60562629e-01 -8.31599608e-02
 -1.42009

## Torch Script Conversion and Saving

In [14]:
input_example = torch.tensor(y_target)
traced_script_module = torch.jit.trace(model, input_example)
test= traced_script_module(y_target)
print(test)

tensor([-2.8384e-03,  2.9917e+00,  5.0038e-04, -4.5768e-01,  2.6327e+00,
         5.2199e-01, -6.8133e-01,  2.3480e+00, -2.7465e-01, -5.8365e-01,
         1.5286e+00, -5.5613e-01, -4.8033e-01,  1.3986e+00, -3.6759e-01,
         2.4042e-02,  4.2311e-01,  5.8777e-02, -2.3321e-01,  4.8493e-01,
        -1.2267e-01,  2.2847e-01,  2.9142e-01,  2.3209e-01,  2.4674e-01,
         9.7857e-03,  3.5545e-01,  1.8088e-01,  7.3289e-02,  4.5366e-01,
         1.0568e-01,  8.6770e-01,  5.2667e-01,  1.6385e-01,  4.5757e-01,
         3.8704e-01,  1.7988e-01, -3.4299e-01,  2.7296e-01,  8.9975e-02,
         3.7762e-01,  1.8984e-01,  7.4218e-03,  3.3323e-01, -1.2592e-01,
        -5.3874e-02,  6.1190e-01, -4.1756e-01, -2.7946e-01, -4.4116e-01,
        -1.9718e-02, -3.1723e-01, -2.6317e-01, -6.5415e-02, -2.6056e-01,
        -8.3160e-02, -1.4201e-01, -2.4519e-01, -1.1053e-01, -2.0157e-01,
        -2.4162e-01, -1.8714e-01, -3.4966e-02,  1.4485e-01, -2.6063e-01,
         1.1956e-01,  5.8964e-01, -1.7848e-01,  2.4

In [15]:
# Save serialized model
traced_script_module.save("CPP_example_model_latest.pt")

In [ ]:
# Test the torch script
test_input= torch.tensor([0, 2, 0.5])
output_example = traced_script_module(test_input)
print(output_example[-12:])
print(original)

In [8]:
#Save Model

if len(losses) == epochs*(samplenum):
    torch.save(model.state_dict(), 'Trained_Model_300420_300s_100e_onlyZpos.pt')
    print('Model saved')
else:
    print('Model has not been trained. Consider loading a trained model instead.')

Model saved
